In [1]:
import logging

from shared_code.utility.spark.set_environ import set_azure_env

set_azure_env()

from shared_code.utility.scripts.reddit_collector import RedditDataCollector

logging.basicConfig(level=logging.INFO)
logging.getLogger("azure.storage").setLevel(logging.WARNING)
logging.getLogger("diffusers").setLevel(logging.WARNING)
logging.getLogger("azure.core").setLevel(logging.WARNING)

In [2]:


import pandas

import dask.dataframe as dd

from shared_code.utility.spark.set_environ import set_azure_env

set_azure_env()


In [3]:
from tqdm.dask import TqdmCallback

cb = TqdmCallback(desc="global")
cb.register()

from tqdm import tqdm

tqdm.pandas()
tqdm.pandas(desc="global")

In [4]:
from tqdm import tqdm

class InnerProgressBar(tqdm):
	def __init__(self, total, desc):
		super().__init__(desc=desc)
		self.total = total
		self.current = 0

	def update_to(self):
		self.update(self.current)

In [5]:
table_name = "training"
image_out_dir = "/data/images/"

In [6]:
subreddits = "oldladiesbakingpies+fatsquirrelhate+trippinthroughtime+spaceporn+memes+EarthPorn+CityPorn+sfwpetite+SFWNextDoorGirls+SFWRedheads+RealGirls_SFW+amihot+SFWcurvy"
subs = subreddits.split("+")

In [7]:
def download_subreddit_images(subreddit) -> int:
	collector: RedditDataCollector = RedditDataCollector(image_out_dir=image_out_dir, table_name=table_name)
	progress.update()
	try:
		count = collector.download_subreddit_images(subreddit)
		return count
	except Exception as e:
		logging.error(f"Error downloading images for {subreddit}: {e}")
		return 0

In [8]:
logging.getLogger("azure.storage").setLevel(logging.WARNING)

data = []

ddf = dd.from_pandas(pandas.DataFrame(subs), npartitions=len(subs))

progress = InnerProgressBar(total=len(subs), desc="global")

for sub in subs:
	try:
		num_images_collected = download_subreddit_images(sub)
		result = {
			"subreddit": sub,
			"count": num_images_collected
		}
		data.append(result)
	except Exception as e:
		logging.error(f"Error downloading images for {sub}: {e}")
		continue

display(pandas.DataFrame(data=data))

global:   8%|▊         | 1/13 [02:02<24:30, 122.58s/it]ERROR:root:Error downloading images for oldladiesbakingpies: The table specified does not exist.
RequestId:8d153d92-d002-0068-7a8b-59b9ff000000
Time:2023-03-18T11:19:16.0378473Z
ErrorCode:TableNotFound
Content: {"odata.error":{"code":"TableNotFound","message":{"lang":"en-US","value":"The table specified does not exist.\nRequestId:8d153d92-d002-0068-7a8b-59b9ff000000\nTime:2023-03-18T11:19:16.0378473Z"}}}

KeyboardInterrupt

